In [ ]:
!pip install -q 'git+https://github.com/drscook/voting_predictor'

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [7]:
import numpy as np
def listify(X):
    """Turns almost anything into a list"""
    if (X is None) or (X is np.nan) or (X==''):
        return []
    elif isinstance(X, str):
        return [X]
    elif isinstance(X, dict):
        return [listify(X.keys()), listify(X.values())]
    else:
        try:  # lists, tuples, sets, maybe more?
            return list(X)
        except:
            return [X]

def capital(X, mode='lower'):
    print(X)
    modes = ['lower', 'capitalize', 'casefold', 'swapcase', 'title', 'upper']
    assert mode in modes, f'mode must one of {modes} ... got {mode}'
    if (X is None) or (X is np.nan) or (X==''):
        return ''
    elif isinstance(X, str):
        return getattr(X, mode)()
    elif isinstance(X, dict):
        return capital(listify(X), mode)
    else:
        try:  # lists, tuples, sets, maybe more?
            return [capital(x, mode) for x in X]
        except:
            return X

# w=['a','b']
# w.lower()
s = ('A', ['b', 'C', {'a':1, 'B':'WEH'}], 4)
# s = {'a':1, 'B':'WEH'}
# s = ''
# listify(s)
# s = 'A'
capital(s)
# listify(s)

('A', ['b', 'C', {'a': 1, 'B': 'WEH'}], 4)
A
['b', 'C', {'a': 1, 'B': 'WEH'}]
b
C
{'a': 1, 'B': 'WEH'}
[['a', 'B'], [1, 'WEH']]
['a', 'B']
a
B
[1, 'WEH']
1
WEH
4


['a', ['b', 'c', [['a', 'b'], [1, 'weh']]], 4]

In [27]:
import numpy as np, pandas as pd
def to_numeric(ser):
    """converts columns to small numeric dtypes when possible"""
    dt = str(ser.dtype)
    if  dt in ['geometry', 'timestamp'] or dt[0].isupper():
        return ser
    else:
        return pd.to_numeric(ser, errors='ignore', downcast='integer')  # cast to numeric datatypes where possible

def listify(X):
    """Turns almost anything into a list"""
    t = type(X)
    if X in [None, np.nan, '']:
        return []
    elif t in [list, tuple, set, pd.Index]:
        return list(X)
    elif t in [dict]:
        return [list(X.keys()), list(X.values())]
    elif t in [np.ndarray]:
        return X.tolist()
    elif t in [pd.Series, pd.DataFrame]:
        return X.values.tolist()
    else:
        return [X]

def prep(X, mode='lower', fix_names=True):
    modes = ['lower', 'capitalize', 'casefold', 'swapcase', 'title', 'upper']
    assert mode in modes, f'mode must one of {modes} ... got {mode}'
    t = type(X)
    if X in [None, np.nan, '']:
        return None
    elif t in [str]:
        return getattr(X.strip(), mode)()
    elif t in [list, tuple, set, pd.Index]:
        return t((prep(x, mode) for x in X))
    elif t in [dict]:
        return dict(zip(*prep(listify(X), mode)))
    elif t in [np.ndarray]:
        return np.array( prep(listify(X), mode))
    elif t in [pd.DataFrame]:
        idx = len(X.index.names)
        X = X.reset_index()
        if fix_names:
            X.columns = prep(X.columns, mode)
        return X.apply(to_numeric).convert_dtypes().set_index(X.columns[:idx].tolist())
    elif t in [pd.Series]:
        return prep(X.to_frame(), mode).squeeze()
    else:
        return X

df = pd.DataFrame()
df['A']=[1,2,3]
df['b']=[1,2,3]
X = df.columns
t = type(X)
X, t(X)
t in [pd.Index]
# X = pd.Series(df)#.squeeze()
# X = df.squeeze()
# X = {'A':4, 5:'B', 'F':['p', 'W']}
# # X='HI'
# # dict(zip(*prep(listify(X))))
# # list(zip(*prep(listify(X))))
# # X = ({'A', 5}, 4, 'b')
# prep(X)
list(X)
# df.to_list()
list(df)

['A', 'b']

In [21]:
X = np.array([[5,6], [1,2]])
for x in X:
    
X
# t = type(X)
# if t in [np.ndarray]:
#     t = np.array
# t(X)
#     # print(4)

array([[5, 6],
       [1, 2]])

In [37]:
dict(zip(*[['a','b', 'c'], [1,2,3]]))

{'a': 1, 'b': 2, 'c': 3}

In [17]:
import numpy as np
x = np.array([2,3,4])
t = type(x)
# y = list(x)
y = x.tolist()
t
# t(y)
t(y)
# t([2,3,4]).shape
t

numpy.ndarray

In [5]:
df = pd.DataFrame()
df['A']=[1,2,3]
# df['b']=[1,2,3]

X = df.squeeze()
type(X)
# X.to_list()
# X = X.values.tolist()
X.to_frame()
# isinstance(X, (pd.Series, pd.DataFrame))


,A
0,1
1,2
2,3


In [47]:
s = 'hi'
s.lower(), s.capitalize(), s.casefold(), s.title(), s.upper(), s.swapcase()
getattr(s, 'lower')()
s['lower']

TypeError: ignored

In [ ]:
%reload_ext google.colab.data_table
%reload_ext autotime
%reload_ext autoreload
%autoreload 2

root_path = '/content/drive/'
local = False
# local = True
if local:
    import sys, google.colab
    google.colab.drive.mount(root_path)
    sys.path.insert(0, root_path + 'MyDrive/helpers')
    sys.path.insert(0, root_path + 'MyDrive/voting_predictor')
    root_path += 'MyDrive/gerrymandering/2022-12/'
# else:
#     !pip install -q --force-reinstall --no-deps 'git+https://github.com/drscook/helpers' 'git+https://github.com/drscook/voting_predictor'

from helpers.common_imports import *
from helpers import utilities as ut
import voting_predictor as vp

self = vp.Redistricter(
    census_api_key = 'dccb7bb4b7df5dff59d2d99c859016f973197e4e',
    bg_project_id = 'redistricting-361203',
    state = 'TX',
    root_path = root_path,
)

self.get_crosswalks()
self.get_assignments()
self.get_shapes()
# self.combine()
# vp.test.f(5)
# ut.

'shapes.TX'

In [ ]:
# df.geometry = df.geometry.buffer(0).apply(orient, args=(1,))
self.bq.df_to_tbl(df, tbl)

'shapes.RI'

In [ ]:
@codetiming.Timer()
def f():
    return 2

f()

Elapsed time: 0.0001 seconds


2

In [ ]:
levels = {
    'state':2,
    'county':3,
    'tract':6,
    'block_group':1,
    'blk':4,
}
X = df.copy()
w = [ut.rjust(df[f'{l}_2010'], d) for l, d in levels.items() if l!='block_group']
display(w[0]+w[1]+w[2]+w[3])
# type(w[0])
# w = df['blk_2010']
# w
# isinstance(w, pd.Series)
# type(w)

index
0          050910209001225
1          050910210007000
2          050910210007031
3          050910210007031
4          050910210008022
                ...       
1036388    485079503025022
1036389    485079503025023
1036390    485079503025024
1036391    485079503025025
1036392    485079503025026
Length: 1036393, dtype: object

time: 5.34 s (started: 2022-12-24 08:31:37 +00:00)


In [ ]:
!pip install --force-reinstall 'git+https://github.com/drscook/REU_temp'
from temp import ut

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/drscook/REU_temp to /tmp/pip-install-hq5yk4dk/reu-temp_63a2d4ef244e4081b4596b78132f1c58
  Running command git clone -q https://github.com/drscook/REU_temp /tmp/pip-install-hq5yk4dk/reu-temp_63a2d4ef244e4081b4596b78132f1c58
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 43 kB 2.3 MB/s 
  Created wheel for REU-temp: filename=reu_temp-0.1.0-py2.py3-none-any.whl size=1147 sha256=c27bd9700e60edac66fd2cbfb5e7193d9d8941b5c58154478497360c4cd0dd80
  Stored in directory: /tmp/pip-ephem-wheel-cache-4c_c20og/wheels/e8/c2/b9/7e6d3335fac2bc78dd184be935c3dd2759267588fa21e58746
  Created wheel for shutils: filename=shutils-0.1.0-py3-none-any.whl size=3516 sha256=e48f5d154b17004e84010db0205dc9443140c4e6e0fcb88085820339640a6817
  Stored in directory: /root/.cac

In [ ]:
from REU_temp import ut
ut.f(45)

ModuleNotFoundError: ignored

In [ ]:
!pip install --force-reinstall 'git+https://github.com/drscook/test_drscook'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/drscook/test_drscook to /tmp/pip-req-build-_4ki3aq6
  Running command git clone -q https://github.com/drscook/test_drscook /tmp/pip-req-build-_4ki3aq6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Cloning https://github.com/drscook/test_drscook2 to /tmp/pip-install-zkbpzjqp/test-drscook2_f132ce5346b34e23ad939165b6c69d55
  Running command git clone -q https://github.com/drscook/test_drscook2 /tmp/pip-install-zkbpzjqp/test-drscook2_f132ce5346b34e23ad939165b6c69d55
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 43 kB 2.1 MB/s 
  Created wheel for test-drscook: filename=test_drscook-1.0.0-py2.py3-none-any.whl size=2369 sha256=1af4c507f7347e1ef76c52c1b9db32f0d3db26205

In [ ]:
# !pip install ipython-autotime
!pip install jupyter-autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 18.0 MB/s 


In [ ]:
%load_ext autotime
import time
for i in range(10):
    print(i)
    time.sleep(.1)

0
1
2
3
4
5
6
7
8
9
